# LAB: KNN 

## 1. Introducción

El objetivo de este LAB es clasificar si un determinado tipo de vino es de alta o baja calidad. Para eso usaremos un dataset que contiene un set amplio de features vinculados a diversas características del vino, tales como acidez, azúclar, densidad, ph, si es tinto, etc.Para llevar a cabo esta tarea ,como input vamos a usar las features de cada vino

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

import matplotlib.pyplot as plt
%matplotlib inline


df = pd.read_csv("../Data/wine.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df['high_quality'].value_counts(normalize=True)

## 2. Clasificando los vinos de alta calidad
- En el conjunto de datos, la calidad esta expresada en una columna numerica. Si quisieramos resolver un problema de regresión podríamos tratar de predecir dicho valor.
    
- Dado que vamos a resolver un problema de clasificación, vamos a predecir "high quality" como una variable binaria que toma valores 0 ó 1.

### 2.1 Realizar un modelo de K-Nearest Neighbors

- Construir la matriz de features.
- Construir la variable target: `high_quality`
- Seleccionar los atribiutos que podrian ser los mejores para el modelo
- Evaluar la accuracy del modelo mediante Cross-Validation
- Experimentar con diferentes valores para el hiperparámetro K y evaluar la performance
- ¿Es necesario estandarizamos las variables?

** Pista:** pueden tratar de generar una función que evalúe diferentes valores del hiperparámetro K.

In [ ]:
from sklearn.model_selection import KFold

def scores_knn(X,y,start,stop,step):
    
    # Voy a querer graficar los distintos valores del score de cross validation en función del hiperparámetro n_neighbors
    # Para esto voy a generar una lista de diccionarios que después se puede convertir fácilmente en DataFrame
    # Lista de Diccionarios
    
    scores_para_df = []
    
    # range(1,40,5) son los posibles valores de n_neighbors que quiero explorar. Son todos los enteros desde el 1
    # hasta el 40 en saltos de a 5. (Start,Stop,Step)
    
    for i in range(start,stop,step):
        # En cada iteración instanciamos el modelo con un hiperparámetro distinto
        
        model = KNeighborsClassifier(n_neighbors=i)

        # cross_val_scores nos devuelve un array de 5 resultados, uno por cada partición que hizo automáticamente CV  
        
        kf = KFold(n_splits=10, shuffle=True, random_state=10)
        
        cv_scores = cross_val_score(model, X, y, cv=kf)

        # Para cada valor de n_neighbours, creo un diccionario con el valor de n_neighbours y la media y el desvío de los scores.
        
        dict_row_score = {'score_medio':np.mean(cv_scores),'score_std':np.std(cv_scores),'n_neighbours':i}

        # Guardo cada uno en la lista de diccionarios
        
        scores_para_df.append(dict_row_score)
        
    df_scores = pd.DataFrame(scores_para_df)
    df_scores['limite_inferior'] = df_scores['score_medio'] - df_scores['score_std']
    df_scores['limite_superior'] = df_scores['score_medio'] + df_scores['score_std']
    return df_scores

In [ ]:
# Construimos la matriz de features, subseteando el DataFrame para que no contenga el campo high_quality (es nuestro target)

X = df[['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol']]
y = df['high_quality']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y)

# Estandarizar
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
df_scores= scores_knn(X_train, y_train, 1, 21, 1)

In [ ]:
# En este caso, la estandarización ayuda algo porque las variables estaban en unidades muy distintas.

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(df_scores['n_neighbours'],df_scores['limite_inferior'],color='r')
plt.plot(df_scores['n_neighbours'],df_scores['score_medio'],color='b')
plt.plot(df_scores['n_neighbours'],df_scores['limite_superior'],color='r')
plt.ylim(0.7, 1);

In [ ]:
df_scores.loc[df_scores.score_medio == df_scores.score_medio.max()]

In [ ]:
best_k = df_scores.loc[df_scores.score_medio ==\
                               df_scores.score_medio.max(),'n_neighbours'].values
best_k = best_k[0]
best_k

In [ ]:
# Elegimos el modelo óptimo que nos había indicado cross validation
model = KNeighborsClassifier(n_neighbors=best_k)

# Lo ajustamos sobre datos de entrenamiento
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Obtenemos la matriz de confusión
confusion_matrix(y_test, y_pred)

In [ ]:
# Computamos el accuracy score:

accuracy_score(y_test, y_pred)